# Prosjekt: Simulering av Antibiotikaresistente Bakterier

Målet med dette prosjektet er å simulere evolusjon av antibiotika resistente bakterier og gjenskape et eksperiment gjort ved *Harvard Medical School*. Forklaringen av eksperimentet er utgangspunktet for modellen som skal brukes i dette prosjektet, så det er nødvendig å se videoen under før du går videre:

<iframe width="560" height="315" src="https://www.youtube.com/embed/plVk4NVIUh8" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

Når du har løst prosjektet, vil du ha produsert simuleringer som gir resultater som i {numref}`bacteria`.


```{figure} ./animations/antibiotika_resistente_bakterier_xceller_300_yceller_150_niter_5000_ensidig.gif
---
name: bacteria
---

Animasjon av evolusjon av bakterier. Den rektangulære platen er delt opp i 5 soner med økende konsentrasjonsgrad med antibiotika. 
```

## Prosjektbeskrivelse

Du skal utvikle en Python-kode som simulerer bakterier i en petriskål der det er økende konsentrasjonsgrad av antibiotika i ulike soner, slik som i eksperimentet ved Harvard Medical School. Numeriske simuleringer har i stor grad fått stor plass som en form for eksperiment der man tester ut modeller med simuleringer for å studere egenskapene til modellen og sjekke om de samsvarer med observasjoner i naturen. Relatert til evolusjonsteorien, er det spesielt to betingelser som må være tilstede for at genforrådet til en art skal endre seg slik at vi observerer evolusjon:

1. Genetisk variasjon. Vi trenger et genforråd som har eller kan skape gener som gir forskjellige egenskaper hos arten. Mutasjon er en viktig komponent her, siden genforrådet kan endres ved at DNA til individer i en populasjon muterer og dermed endres.
2. Variasjon i miljø. Vi trenger et miljø som arten lever i der noen gener gir individer egenskaper som gir de en fordel med tanke på å overleve og reprodusere seg over andre individer i arten. 

Som mål skal du blant lage en animasjon og en figur som kartlegger populasjonsveksten over tid.

## Modell for bakterien

Her skal du sette opp modellen for bakterien og legge grunnlaget for simulering ved å skrive mindre funksjoner som løser deler av problemet. Til slutt skal du sy det sammen og simulere et system av mange bakterier. 

### En modell for bakterien og kode for modellen

Tenk deg at bakteriene levere i på en rektangulær plate. Tenk deg at vi deler opp langsiden i $L$ deler og kortsiden i $K$ deler. Tilsammen kan vi da tenke oss at det finnes $L \times K$ ruter som vi kan plassere en bakterie på. 

Når man skal sette lage en modell for et fysisk system, er første steg å forenkle og velge ut relevante variabler som vi tror har en effekt på hvordan systemet fungerer. I dette tilfellet er det fysiske systemet en bakterie. Hvis vi tenker oss om, bør vi i det minste passe på å ha en modell som fanger opp vanlige egenskaper hos en bakterie:

1. **Bevegelse**: Modellen for bakteriene må sørge for at bakteriene kan flytte seg rundt på en måte som replikerer bevegelsesmønstre til bakterier.
2. **Antibiotikaresistens**: Modellen for bakterien må ha en måte å representere antibiotikaresistens på.
3. **Dødsfall**: Bakteriene er nødt til å kunne dø i populasjonen.
4. **Reproduksjon**: Modellen for bakteriene må ta hensyn til at bakteriene kan reprodusere gjennom ukjønnet formering.
5. **Mutasjon**: Bakteriene må kunne undergå genetiske mutasjoner slik at genetisk variasjon kan forekomme i populasjonen av bakterier.

### Bevegelse

Modellen for bakteriene må sørge for at forflytningen til bakteriene reflekterer hvordan **ekte** bakterier beveger seg. I praksis, vil bevegelsesmønsteret til bakterier opptre som svært tilfeldig. En enkel modell for bevegelsen til bakterien er derfor å generere hvilken retningen den går som **helt tilfeldig**. Det viser seg at dette garanterer at bakterien vil besøke alle mulige posisjoner på den rektangulære platen, et resultat som følger fra den matematisk teorien for **virrevandring**. Du kan derfor flytte bakteriene rundt helt tilfeldig. 

Siden bakterien lever på en rektangulær plate med $n_x$ posisjoner langs langsiden og $n_y$ posisjoner langs kortsiden, vil bakterien kunne leve på $n_x \times n_y$ ruter/celler i "Universet" til bakteriene. La oss kalle langsiden for $x$-retning og kortsiden for $y$-retning. Da kan bakterien ha posisjoner langs $x$-aksen med verdiene 

$$
x = 0, 1, 2, 3, \ldots, n_x - 1,
$$

og langs $y$-aksen 

$$
y = 0, 1, 2, 3, \ldots, n_y - 1.
$$

Vi kan derfor assosiere et koordinat $(x, y)$ som en beskrivelse av posisjonen til en bakterie på den rektangulære platen. 

Det er naturlig å benytte seg av et objekt i Python som heter *dictionary*, med datatype `dict`, som kan lagre denne informasjonen på følgende måte:

In [2]:
bakterie = {
    "x": 2,
    "y": 3,
}

I eksemplet her er `bakerie` plassert i koordinatet $(2, 3)$. Med andre ord er $x = 2$ og $y = 3$.
Hvis vi antar at bakterien **kun** kan flytte seg én plass av gangen i $x$- og $y$-retning, vil vi kunne flytte bakterien på følgende måte:

In [3]:
import numpy as np # bibliotek for numerisk matte
bakterie["x"] += np.random.randint(-1, 2) # Trekker tilfeldig en av tallene -1, 0 eller 1.
bakterie["y"] += np.random.randint(-1, 2)
print(bakterie)

{'x': 2, 'y': 3}


Vi kan også hente ut dataen som er lagret i `bakterie` slik:

In [4]:
x = bakterie.get("x")
y = bakterie.get("y")

print(f"{x = }")
print(f"{y = }")

x = 2
y = 3


#### Modell for antibiotikaresistens

Vi må ha en måte å inkludere antibiotikaresistens i modellen for bakteriene. Vi skal gå for en enkel løsning der vi modellerer en bakterie sin grad av antibiotikaresistens som et tall mellom 0 og 1, der 0 representerer *ingen* antibiotikaresistens og 1 representerer 100% antibiotikaresistens.

Vi kan derfor utvide måten vi lagrer informasjon om bakterien. En bakterie med ingen antibiotikaresistens kan for eksempel lages slik:


In [5]:
bakterie = {
    "x": 2,
    "y": 3,
    "resistens": 0,
}

På samme måte som før, kan vi både endre og hente ut antibiotikaresistensen fra `bakterie`:

In [6]:
bakterie["resistens"] += 0.1 # Øker antibiotikaresistens med 0.1
resistens = bakterie.get("resistens") # Henter ut antibiotikaresistens fra bakterien 
print(f"{resistens = }")

resistens = 0.1


#### Konsentrasjonsgrad av antibiotika og effekten på bakterien

Vi kan modellere konsentrasjonsgrad av antibiotika på akkurat samme måte som vi modellerer antibiotikaresistens hos bakterien. Lar vi konsentrasjonen være et tall mellom 0 og 1 på ulike områder på den rektangulære platen, akkurat som i eksperimentet ved Harvard Medical School, så får vi gjenskapt miljøvariasjonen de hadde. Vi trenger i praksis da en funksjon som har såkalt *delt forskrift*. La konsentrasjon være en funksjon $K(x)$. Da kan den være på formen

$$
K(x) = 
\begin{cases}
    0, \qquad 0 \leq x < 0.2 n_x, \\
    0.25, \qquad 0.2 n_x \leq x < 0.4 n_x, \\
    0.5, \qquad 0.4 n_x \leq x < 0.6 n_x, \\
    0.75, \qquad 0.6 n_x \leq x < 0.8 n_x, \\
    0.99, \qquad 0.8 n_x \leq x < n_x, \\
\end{cases}
$$
der $x$ er $x$-koordinaten til bakterien.
Da vil vi ha **fem soner** med økende grad av antibiotikakonsentasjon. For å sjekke hvilken konsentrasjon en gitt bakterie lever i, kan du bruke en Pythonfunksjon som tar inn bakterien sin $x$-koordinat og returnerer antibiotikakonsentrasjonen.

*For å lage en slik funksjon, kan du ta utgangspunkt i kodeskallet under. Du må fylle inn der det står `NotImplemented`*

In [ ]:
def antibiotika_konsentrasjon(x, nx):
    if 0 <= x < 0.2*nx:
        return 0
    elif 0.2*nx <= x < 0.4*nx:
        return 0.25
    elif NotImplemented:
        return NotImplemented
    elif NotImplemented:
        return NotImplemented
    else:
        return NotImplemented

````{dropdown} Løsningsforslag

```python
def antibiotika_konsentrasjon(x, nx):
    if 0 <= x < 0.2*nx:
        return 0
    elif 0.2*nx <= x < 0.4*nx:
        return 0.25
    elif 0.4*nx <= x < 0.6*nx:
        return 0.5
    elif 0.6*nx <= x < 0.8*nx:
        return 0.75
    else:
        return 0.99
```

````

### Hvordan dør bakteriene?

Bakterier har ingen form for alderdom, så i en modell for bakterien, holder at vi sjekker om antibiotikaresistensen tar livet av bakterien eller ikke. 
La $r$ være antibiotikaresistensen til en bakterie og $x$ være den posisjon, og $K(x)$ være konsentrasjon av antibiotika i $x$. En logisk måte å modellere hvorvidt en bakterie skal dø er slik:

1. Dersom konsentrasjonen er mindre enn eller lik resistensen, så overlever bakterien uansett.
2. Dersom konsentrasjonen er større enn resistensen, så har bakterien en viss sjanse for å overleve og en viss sjanse for å dø. Jo større forskjellen er mellom resistensen og konsentrasjonen, jo høyere er sjansen for å dø. 

Punkt nr. 2 kan oppnås ved å benytte seg av noe som kalles for **slice sampling**. 

```{prf:algorithm} Bakteriedrap
:nonumber:

**Input** Antibitiokaresistens $r$, posisjon $x$


1. Regn ut konsentrasjon av antibiotika i $x$ med $K(x)$.
2. Hvis $r >= K(x)$, så overlever bakterien.
3. Hvis $r < K(x)$:
    - Trekk et tilfeldig tall $u$ mellom 0 og 1.
    - Dersom $u \geq K(x) - r$, så overlever bakterien. Hvis ikke dør den.
```

*Du ta utgangspunkt i kodeskallet for en funksjon `bakteriedrap` som implementerer denne algoritmen. Du må fylle inn der det står `NotImplemented`.*

In [8]:
def bakteriedrap(bakterie):
    """Tar inn en bakterie sjekker om den skal dø. 
    Returnerer `False` hvis bakterien skal overleve, og `True` hvis den skal dø.
    """
    x = bakterie.get("x")
    resistens = bakterie.get("resistens")
    if resistens >= antibiotika_konsentrasjon(x):
        return False # Returner false, ikke drep bakterien!
    else:
        u = np.random.uniform() # tilfeldig tall mellom 0 og 1.
        if NotImplemented:
            return NotImplemented
        else:
            return NotImplemented

### Reproduksjon

En egenskap vi må ta hensyn til i en populasjon av bakterier er at når antall bakterier i populasjonen blir svært høy (og nærmer seg det maksimale antalle individer kalt **bæreevnen** til populasjonen), så vil sannsynligheten for at en bakterie i populasjon vil ha reprodusere bli svært lav. Dersom det er få individerer i populasjonen, derimot, vil sannsynligheten for reproduksjon av en tilfeldig bakterie være høy. Vi forventer at populasjonen følger en logistisk vekst, og vi kan sørge for at populasjonsveksten tilfredstiller en slik modell evd å benytte en sannsynlighetsfordeling for at en tilfeldig bakterie reproduserer ved

$$
p(n) = 2\left( 1 - \frac{1}{1 + e^{-b\cdot n}} \right), \qquad n \geq 0,
$$

der $n$ er antall individerer i bakteriepopulasjonen og $b$ er en parameter som bestemmer bæreevnene til populasjonen.

#### Hvordan sjekke om en bakterie kan reprodusere?

For å sjekke om en bakterie kan reprodusere, må du hente ut hvor mange bakterier som er i populasjonen. Vi skal se på hvordan vi gjør det senere. For nå, la oss bare anta at antall bakterier i populasjonen er $n$. For å sjekke om en bakterie for reprodusere, trekker vi da et tilfeldig tall $u$ mellom $0$ og $1$ (som vi kan gjøre med `u = np.random.uniform()`). Da gjelder følgende:

1. Hvis $p(n) > u$ så får bakterien reprodusert.
2. Hvis ikke er det for vanskelig for bakterien å reprodusere, og ingen reproduksjon vil skje.

### Hvordan lagre flere bakterier?

I lys av hva vi har sett på så langt, er det naturlig å lure på hvordan vi skal lagre mange bakterier av gangen. Tross alt består jo en populasjon av bakterier av flere enn én bakterie. Python bidrar med en nyttig datatype kalt for en *liste*, eller `list`, som gir oss evnen til å lagre flere ting i ett og samme objekt. 

For eksempel, kan vi vi lagre to bakterier på følgende vis:

In [4]:
bakterier = [
    {"x": 1, "y": 3}, # bakterie 1 på koordinat (1, 3)
    {"x": 3, "y": 5}, # bakterie 2 på koordinat (3, 5)
]